In [5]:
import pandas as pd
import os
import json
import requests

In [6]:
sub_data = pd.read_excel("../data/sampled_df_with_generated_questions.xlsx")

In [7]:
def gpt_4_1(headers, prompt):

    url = "https://apim.stanfordhealthcare.org/openai-eastus2/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview"
    payload = json.dumps({
        "model": "gpt-4.1", 
        "messages": [{"role": "user", "content": prompt}]
    })
    response = requests.request("POST", url, headers=headers, data=payload)
    message_content = response.json()["choices"][0]["message"]["content"]
    return message_content


In [8]:
def gemini_2_5_pro(headers, prompt):
    url = "https://apim.stanfordhealthcare.org/gemini-25-pro/gemini-25-pro"
    payload = json.dumps({
        "contents": [{"role": "user", "parts": [{"text": prompt}]}]
    })
    response = requests.request("POST", url, headers=headers, data=payload)
    message_content = response.json()[0]['candidates'][0]['content']['parts'][0]['text']
    return message_content

In [9]:
my_key = os.getenv("HEALTHREX_API_KEY")
# Common Headers (Used for all models)
headers = {'Ocp-Apim-Subscription-Key': my_key, 'Content-Type': 'application/json'}
def parser_LLM(prompt, model = "gpt-4.1", 
             headers = headers):
    message_content = ""
    if model == "gpt-4.1":
        message_content = gpt_4_1(headers, prompt)
    if model == "gemini-2.5-pro":
        message_content = gemini_2_5_pro(headers, prompt)
    return message_content
def evaluator_LLM(evaluation_prompt, model = "gpt-4.1", headers = headers):
    message_content = ""
    if model == "gpt-4.1":
        message_content = gpt_4_1(headers, evaluation_prompt)
    if model == "gemini-2.5-pro":
        message_content = gemini_2_5_pro(headers, evaluation_prompt)
    return message_content





In [10]:
def prompt_preparation(result, actual_response, subject, LLM_response):
    result_json = json.loads(result)
    result_json["message_subject"] = subject
    result_json["LLM-generated_response"] = LLM_response
    result_json["actual_response"] = actual_response
    evaluation_prompt = evaluator_LLM(result_json)
    return evaluation_prompt


In [25]:
patient_message = sub_data["Patient Message"][10]

In [26]:
actual_response = sub_data["Actual Response Sent to Patient"][10]

In [27]:
notes = sub_data["Prompt Sent to LLM"][10]

In [28]:
subject = sub_data["Subject"][10]

In [15]:
LLM_response = sub_data["Suggested Response from LLM"][10]

In [16]:
def prompt_preparation(notes, patient_message):
  prompt = f"""You are an expert clinical data extractor. Given the unstructured clinical information below, extract and parse it into the structured JSON format provided.

  **Follow these rules carefully:**

  - **Field Fidelity:** Populate each field using only the explicit information provided in the clinical data. Use “Unknown” or "" for missing or unclear fields, per the template.
  - **Information Granularity:** For list fields (e.g., "history_of_present_illness", "past_medical_history"), enter each bullet, sentence, or clinically distinct concept as a separate item.
  - **Relevance:** Include all clinically relevant complaints or concerns discussed, including those introduced in direct messages by the patient, as symptoms/chief complaints or in new "assessment_and_plan" issues.
  - **Physical Exam:** Record each PE subfield as completely as possible. If side-specific findings are present (e.g., right/left ear), include these in the most granular field appropriate.
  - **Assessment and Plan:** Enter each active issue, including newly raised complaints from the patient, along with provider instructions, recommended follow-up, or referral steps. If a complaint is new (e.g., from a patient message, not the prior note), include your clinical response as an entry.
  - **Instructions:** General instructions (e.g., when to follow up, how to schedule) should be recorded in "general_guidelines"; pharmacy details as specified.
  - **Patient Message:** Always copy the patient’s message verbatim.
  - **Additional Notes:** Include any clinical details, context, or provider action plans not clearly fitting in the other structured fields.

  **Strict Guidelines:**
  - Do not infer or hallucinate any data not clearly present.
  - Do not summarize or condense the patient's clinical complaints or history; preserve their language and details in the output.
  - Fields with multiple possible entries (e.g., medications, history, complaints) should be output as complete arrays.

  ### Clinical Information:  
  {notes}

  ### Structured JSON Template:
  {{
    "provider_info": {{
      "provider_name": "",
      "department_specialty": "",
      "department_name": "",
      "department_phone": "",
      "primary_care_provider": ""
    }},
    "patient_info": {{
      "patient_name": "",
      "patient_age": ""
    }},
    "visit_info": {{
      "visit_date": "",
      "visit_type": "",
      "location": {{
        "patient": "",
        "provider": ""
      }},
      "chief_complaint": "",
      "history_of_present_illness": [],
      "active_problems": [
        {{
          "problem": "",
          "code": ""
        }}
      ],
      "past_medical_history": [
        {{
          "condition": "",
          "diagnosed": "",
          "medication": "",
          "note": ""
        }}
      ],
      "physical_exam": {{
        "general": "",
        "HEENT": "",
        "respiratory": "",
        "neurological": "",
        "cardiovascular": "",
        "gastrointestinal": "",
        "musculoskeletal": "",
        "skin": "",
        "psych": ""
      }},
      "assessment_and_plan": [
        {{
          "issue": "",
          "instructions": []
        }}
      ]
    }},
    "instructions": {{
      "general_guidelines": [],
      "pharmacy_info": {{
        "default_pharmacy": {{
          "name": "",
          "address": "",
          "phone": "",
          "fax": ""
        }}
      }}
    }},
    "additional_notes": "", 
    "patient_message": {patient_message}
  }}

  Respond ONLY with the completed JSON. No additional explanation or commentary."""
  return prompt


In [17]:
def evaluation_prompt_preparation(result_json):
  evaluation_prompt = f"""
  Comprehensive Evaluation Prompt
  You are a clinical quality-assessment assistant tasked with evaluating responses generated by an AI model to patient messages.

  Your evaluation involves:

  Message Categorization: Classify the patient’s message into a category based on the subject title and the message content.

  Response Evaluation: Provide quantitative scores (0-10) for each defined evaluation dimension.

  Error Identification: Clearly identify specific errors, categorize them (Clinical Error vs Non-Clinical Error), assign a severity score (1-5), and localize the exact text excerpt of each error.

  📌 Scoring Definitions
  Response Quality Dimensions (Score: 0-10):
  Clinical Accuracy (0-10):

  0: Completely incorrect or dangerous advice.

  5: Partially correct but incomplete or with moderate mistakes.

  10: Completely accurate, clinically appropriate, and safe.

  Urgency Recognition (0-10):

  0: Urgency completely unrecognized or incorrectly interpreted, potentially dangerous.

  5: Moderately appropriate urgency recognition, partial omission of critical issues.

  10: Completely correct urgency identification and action advice.

  Professional Consultation Guidance (0-10):

  0: No guidance or completely incorrect guidance.

  5: Partial or incomplete professional guidance.

  10: Excellent and appropriate consultation guidance.

  Sensitivity & Clarity (0-10):

  0: Rude, dismissive, or entirely unclear.

  5: Moderately empathetic, some unclear language or slight dismissiveness.

  10: Fully empathetic, clear, and supportive language.

  Error Severity Definitions (Severity Score: 1-5):
  1: Minor issue unlikely to affect patient care or experience.

  2: Noticeable but unlikely to significantly affect clinical outcome.

  3: Significant error that could lead to patient confusion or delay in care.

  4: Serious error, potentially impacting patient health negatively.

  5: Critical or potentially lethal error requiring immediate correction.

  📑 Structured Output Template
  Return your response strictly in this JSON format:
  {{
    "message_categorization": {{
      "subject": "[provided subject title]",
      "type": "[Appointment Request | Medication Request | Test Result Inquiry | Clinical Advice Request | Referral Request | Administrative Request | General Inquiry| Other (specify)]",
      "reasoning": "[brief reasoning for the chosen category]"
    }},
    "response_evaluation": {{
      "clinical_accuracy": {{
        "score": "[0-10]",
        "reasoning": "[brief reasoning]"
      }},
      "urgency_recognition": {{
        "score": "[0-10]",
        "reasoning": "[brief reasoning]"
      }},
      "professional_consultation_guidance": {{
        "score": "[0-10]",
        "reasoning": "[brief reasoning]"
      }},
      "sensitivity_clarity": {{
        "score": "[0-10]",
        "reasoning": "[brief reasoning]"
      }}
    }},
    "errors_identified": [
      {{
        "type": "[Clinical Error | Non-Clinical Error]",
        "severity": "[1-5]",
        "description": "[brief clear description of the error]",
        "text_excerpt": "[exact problematic text excerpt from response]",
        "error_in_physician_response": "[Yes | No]",
        "reason_for_error_in_physician_response": "[exact text excerpt from actual physician response from the result_json to explain why this error is/isn't in physician response]"
      }}
    ]
  }}

  Task Instructions
  Given the structured data below, perform your evaluation exactly as specified above:
  {result_json}

  Rules:
  Focus solely on evaluating the quality, appropriateness, accuracy, and clarity of the LLM-generated response.

  Do NOT evaluate the physician’s actual response (it's provided only for reference as a ground truth).

  Be precise, objective, and adhere strictly to the provided scoring scales and categories.

  If there are no identifiable errors, return "errors_identified": [].

  Do not generate additional narrative commentary outside the JSON structure.
  """
  return evaluation_prompt


In [29]:
parse_prompt = prompt_preparation(notes, patient_message)
result = parser_LLM(parse_prompt)

In [19]:
print(result)

{
  "provider_info": {
    "provider_name": "Belanger, Ann Marie",
    "department_specialty": "Family Medicine",
    "department_name": "Alameda Family Physicians",
    "department_phone": "510-521-2300",
    "primary_care_provider": "Belanger, Ann Marie"
  },
  "patient_info": {
    "patient_name": "Skylar Howell",
    "patient_age": "29 Y"
  },
  "visit_info": {
    "visit_date": "9/16/2024",
    "visit_type": "Ambulatory Visit",
    "location": {
      "patient": "",
      "provider": ""
    },
    "chief_complaint": "Preventative Care",
    "history_of_present_illness": [
      "Patient presents with Preventative Care",
      "Pt has no other issues or concerns"
    ],
    "active_problems": [
      {
        "problem": "Encounter to establish care",
        "code": ""
      },
      {
        "problem": "Preventative health care",
        "code": ""
      },
      {
        "problem": "Anxiety and depression",
        "code": ""
      },
      {
        "problem": "PTSD (post-tra

In [20]:
evaluation_prompt = evaluation_prompt_preparation(result)
evaluation = evaluator_LLM(evaluation_prompt)
print(evaluation)

{
  "message_categorization": {
    "subject": "Hearing concerns and request for testing after military service",
    "type": "Clinical Advice Request",
    "reasoning": "The patient is expressing concern about worsening hearing, noting a possible connection to military service, and is specifically asking for advice regarding hearing testing."
  },
  "response_evaluation": {
    "clinical_accuracy": {
      "score": "10",
      "reasoning": "The response appropriately addresses the concern by recommending formal audiologic evaluation and provides clear follow-up actions, aligning with standard of care for new/progressive hearing loss."
    },
    "urgency_recognition": {
      "score": "10",
      "reasoning": "The response identifies the urgency, offering explicit instruction to seek urgent in-person evaluation if sudden worsening or severe symptoms develop."
    },
    "professional_consultation_guidance": {
      "score": "10",
      "reasoning": "Provides clear advice for audiology

In [21]:
print(patient_message)

Hi Dr Belanger,I'm reaching out because I forgot to mention a concern that my husband and I both have about my hearing. It seems to be getting worse, fairly quickly, to us & I wonder if it's due to my military service. Could we arrange for me to have testing done? thanks& I look forward to hearing back from you :)-Skylar


In [22]:
print(actual_response.replace("<10>", "\n"))

Hi Skylar, I've ordered this for your to be done at the Hearing Zone here in Alameda, if you prefer Stanford in Emeryville let me know. Dr. Belanger 


In [23]:
print(LLM_response.replace("<10>", "\n"))

Hi Skylar,

Thank you for reaching out. I'm sorry to hear about your concerns with your hearing. Given the rapid changes you're experiencing, it would be a good idea to have this evaluated further. I will review your request and arrange for a referral to an audiologist for comprehensive hearing testing. 

Please keep an eye on your MyHealth account for the referral details. If you have any other questions or need further assistance, feel free to reach out.

Best regards,


In [31]:
result = """2025-06-17 14:10:54,632 - Original Message:
2025-06-17 14:10:54,633 - original_query_message: Hello,I was able to get an appointment for a colonoscopy next Wednesday, 12/11. The directions say to ask my doctor about what to do regarding medications I am taking for diabetes with respect to the procedure. Could you let me know if I should change my medicine schedule in some way? Thank you so much!Emily 
2025-06-17 14:10:56,371 - Query Parameters:
2025-06-17 14:10:56,371 - query_message: Hi there,I just got scheduled for a colonoscopy next Thursday (the 14th), and I wanted to reach out with a question. I have type 2 diabetes and take both Metformin and insulin daily. I’m a bit worried about how the prep for the colonoscopy and not eating much the day before might affect my blood sugar levels. Should I adjust the amount or timing of my diabetes meds while I’m fasting for the procedure? I really want to avoid going too low or too high. Any specific advice would be really appreciated—thanks so much!
2025-06-17 14:10:56,372 - receiver: AFP MYHEALTH POOL
2025-06-17 14:10:56,372 - department: FAMILY MEDICINE AFP ALAMEDA
2025-06-17 14:10:56,373 - specialty: Family Medicine
2025-06-17 14:10:57,218 - 
Number of results: 5
2025-06-17 14:10:57,219 - ################################################################################START################################################################################
2025-06-17 14:10:57,220 - ✅ similarity: 0.7686
2025-06-17 14:10:57,221 - Sender: MYCHART, GENERIC [ T200162] -> the retrieved similar message : Hello,I was able to get an appointment for a colonoscopy next Wednesday, 12/11. The directions say to ask my doctor about what to do regarding medications I am taking for diabetes with respect to the procedure. Could you let me know if I should change my medicine schedule in some way? Thank you so much!Emily 
2025-06-17 14:10:57,222 - Provider's response to this similar message: Hi Emily,For your colonoscopy, you should follow these guidelines regarding your diabetes medications:1. **Metformin (Glucophage XR)**: You can continue taking your metformin as usual up until the day before the procedure. On the day of the procedure, do not take your metformin until after the colonoscopy is completed.2. **Ozempic (semaglutide)**: You can continue taking your Ozempic as usual. Since it is a weekly injection, you do not need to adjust the timing around the procedure.3. **Blood Sugar Monitoring**: Continue to monitor your blood sugar levels closely, especially since you will be fasting. If you notice any significant changes or have concerns, please let us know.4. **Diet and Preparation**: Follow the specific dietary instructions provided by the colonoscopy preparation guidelines. This usually involves a clear liquid diet the day before the procedure and taking a bowel prep solution.If you have any further questions or concerns, please feel free to reach out. Good luck with your procedure!Best regards,Sujata Behera, PA-C, MA, MPH
2025-06-17 14:10:57,224 - -----------printing the whole thread-------------
2025-06-17 14:10:57,229 - Thread ID: 252536125
--------------------------------------------------------------------------------
idx: 144243
Subject: Ordered Test Question
----------------------------------------
Date Sent: 2024-12-06 00:00:00
----------------------------------------
Sender Message:
Hello,I was able to get an appointment for a colonoscopy next Wednesday, 12/11. The directions say to ask my doctor about what to do regarding medications I am taking for diabetes with respect to the procedure. Could you let me know if I should change my medicine schedule in some way? Thank you so much!Emily 
----------------------------------------
Provider Response by AFP MYHEALTH POOL:
Hi Emily,For your colonoscopy, you should follow these guidelines regarding your diabetes medications:1. **Metformin (Glucophage XR)**: You can continue taking your metformin as usual up until the day before the procedure. On the day of the procedure, do not take your metformin until after the colonoscopy is completed.2. **Ozempic (semaglutide)**: You can continue taking your Ozempic as usual. Since it is a weekly injection, you do not need to adjust the timing around the procedure.3. **Blood Sugar Monitoring**: Continue to monitor your blood sugar levels closely, especially since you will be fasting. If you notice any significant changes or have concerns, please let us know.4. **Diet and Preparation**: Follow the specific dietary instructions provided by the colonoscopy preparation guidelines. This usually involves a clear liquid diet the day before the procedure and taking a bowel prep solution.If you have any further questions or concerns, please feel free to reach out. Good luck with your procedure!Best regards,Sujata Behera, PA-C, MA, MPH
----------------------------------------
2025-06-17 14:10:57,233 - ################################################################################END################################################################################
2025-06-17 14:10:57,233 - ################################################################################START################################################################################
2025-06-17 14:10:57,234 - ✅ similarity: 0.6049
2025-06-17 14:10:57,234 - Sender: MYCHART, GENERIC [ T200162] -> the retrieved similar message : Thank you. You indicated in your reply that metabolic panel will test for blood sugars but you didn’t order metabolic panel, only lipid panel if I understand your orders correctly.The lipid panel order says fasting not required. If that’s correct and you don’t want any other tests that require fasting, then I will reschedule lab appointment so results are in, in advance of my doctors visit.Otherwise early morning appointment (fasting) not available until after doctors visit Please confirm either way Don
2025-06-17 14:10:57,235 - Provider's response to this similar message: Hello Don Dr Carper placed another lab order today 11/04/2024 for metabolic panel,hemoglobinA1C,CBC with Diff and PSA make sure when You go to Lab to ask them to do also lipid panel ordered on 11/01/2024 and fast for 12 hoursMarlene Aguilar-Duarte, MA
2025-06-17 14:10:57,236 - -----------printing the whole thread-------------
2025-06-17 14:10:57,238 - Thread ID: 249591209
--------------------------------------------------------------------------------
idx: 293506
Subject: RE: Ordered Test Question
----------------------------------------
Date Sent: 2024-11-04 00:00:00
----------------------------------------
Sender Message:
Thank you. You indicated in your reply that metabolic panel will test for blood sugars but you didn’t order metabolic panel, only lipid panel if I understand your orders correctly.The lipid panel order says fasting not required. If that’s correct and you don’t want any other tests that require fasting, then I will reschedule lab appointment so results are in, in advance of my doctors visit.Otherwise early morning appointment (fasting) not available until after doctors visit Please confirm either way Don
----------------------------------------
Provider Response by AFP MYHEALTH POOL:
Hello Don Dr Carper placed another lab order today 11/04/2024 for metabolic panel,hemoglobinA1C,CBC with Diff and PSA make sure when You go to Lab to ask them to do also lipid panel ordered on 11/01/2024 and fast for 12 hoursMarlene Aguilar-Duarte, MA
----------------------------------------
idx: 293505
Subject: Ordered Test Question
----------------------------------------
Date Sent: 2024-11-04 00:00:00
----------------------------------------
Sender Message:
I have a question about Lipid Panel with Calculated LDL ordered on 11/1/2024. For the past 2 years you have also ordered a  PSA and Metabolic panel tests and in 2022 CBC w/o diff. Are you purposely not scheduling these tests this year?Also, I don't think we've ever tested for Hemoglobin A1c. I read that should be done after age 35 or so. Is that not the case.Lastly, the earliest, early morning (8 AM) lab appointment I could get is after my scheduled annual Medicare physical. If I truly don't need to fast I can move that to a date before the office visit so after answering the above , please confirm no fasting required. 
----------------------------------------
Provider Response by AFP CARE TEAM POOL:
Hi Don, This is Irene Kwan, FNP-C responding on behalf of Dr. John Carper. In general, sometimes certain lab tests are not always ordered on a yearly basis, and only ordered if clinically necessary.  For example, CBC may not be needed if there is no sign of fatigue, weakness, blood loss, history of anemia, or concerning presence of infection. Sometimes Medicare also does not cover for certain tests if deemed not medically necessary or at risk, thus patients may have to pay out of pocket for tests requested by the patient. Your metabolic panel test will check for a fasting blood sugar, and if it is elevated, then we can always check hemoglobin A1c with a fingerstick blood check in office. Your labs ordered on 11/1/2024 are recommended to be fasting so that we can accurately check your glucose level and lipid (cholesterol) levels.  Thank you for your understanding, and should you have further questions about lab tests, it can be discussed at your upcoming appointment. Best, Alameda Family PhysiciansIrene Kwan, FNP-C, on behalf of Dr. John Carper(510)521-2300Please note that MyHealth messages are meant for non-urgent concerns that can be answered by a simple email response.  Messages will be forwarded by the office staff when the office is open, to the appropriate provider.Responses may take up to 3 business days (excluding weekends and holidays)If you feel you need a more timely response, or feel that your concerns are more complicated, please call our office to set up a video or in office visit.  
----------------------------------------
2025-06-17 14:10:57,239 - ################################################################################END################################################################################
2025-06-17 14:10:57,239 - ################################################################################START################################################################################
2025-06-17 14:10:57,240 - ✅ similarity: 0.5817
2025-06-17 14:10:57,240 - Sender: MYCHART, GENERIC [ T200162] -> the retrieved similar message : I am scheduled for a colonoscopy on 12/13 but have not received any prep medications yet. Is that normal? Should I expect a prescription to be sent to my pharmacy soon? Thanks,Simon
2025-06-17 14:10:57,240 - Provider's response to this similar message: Dear Simon,Thank you for reaching out. The gastro office usually sends the prep medication and provides you with procedure instructions. Please contact Dr.Huilan Cheng office, (510)548-6555 . Sincerely,Yazmin Rodriguez, MA
2025-06-17 14:10:57,241 - -----------printing the whole thread-------------
2025-06-17 14:10:57,242 - Thread ID: 252486740
--------------------------------------------------------------------------------
idx: 145918
Subject: Non-urgent Medical Question
----------------------------------------
Date Sent: 2024-12-06 00:00:00
----------------------------------------
Sender Message:
I am scheduled for a colonoscopy on 12/13 but have not received any prep medications yet. Is that normal? Should I expect a prescription to be sent to my pharmacy soon? Thanks,Simon
----------------------------------------
Provider Response by AFP MYHEALTH POOL:
Dear Simon,Thank you for reaching out. The gastro office usually sends the prep medication and provides you with procedure instructions. Please contact Dr.Huilan Cheng office, (510)548-6555 . Sincerely,Yazmin Rodriguez, MA
----------------------------------------
2025-06-17 14:10:57,243 - ################################################################################END################################################################################
2025-06-17 14:10:57,244 - ################################################################################START################################################################################
2025-06-17 14:10:57,244 - ✅ similarity: 0.5744
2025-06-17 14:10:57,245 - Sender: MYCHART, GENERIC [ T200162] -> the retrieved similar message : Hi Dr. Tsao,I got my blood tests results yesterday, and I’m very worried that my Glucose is high. It was slightly outside the normal range for non-fasting, and way outside the normal range for fasting. I didn’t fast but the only thing I ate was oatmeal and eggs and it was about 4 hours before the blood draw so I was very surprised that the glucose level wasn’t closer to the normal fasting range, and I assume it’s related my pancreas function. As you know, I’m very worried about pancreatic cancer.I am eager to get your analysis about it, and any other concerns about other test results, and to move quickly on whatever next step I should take.I am still struggling to eat food and still feeling discomfort after I do eat. Thank you,Ritsuko
2025-06-17 14:10:57,245 - Provider's response to this similar message: HI Ritsuko, I have not been able review your results until now and did not see the email that you had sent. Please note, Your results will release immediately, likely before I can review them.  Once I review your results, which may be up to 3-4 business days, I will send comments regarding the results.    We can check another test to make sure your blood sugar average (A1C) is in normal range as your fasting and non-fasting blood sugar can react to when you ate, what you ate, stress, etc.  Your A1C has been ordered and sent electronically to Labcorp to be done non-fasting.  You do not need to have the paper orders, but you can print them out at home through MyHealth (under tests-> ordered tests).  You should try to schedule an appointment for your test at the lab (online or by phone), as they can be busy.  Your results will release immediately, likely before I can review them.  Once I review your results, which may be up to 3-4 business days, I will send comments regarding the results.    Your white blood cell count was slightly low, but improved from your previous which is usually low.  Your thyroid hormone, electrolytes, pancrease enzymes, liver and kidney function are normal.  Your H.pylori stool test is negative for infection.  Please follow up with Dr. Cheng regarding your GI symptoms. Lilian Li-Ning Tsao, MD, 10/21/2024, 8:39 AMI'm not in the office on Mondays, but check my inbox periodically.  
2025-06-17 14:10:57,245 - -----------printing the whole thread-------------
2025-06-17 14:10:57,247 - Thread ID: 248313136
--------------------------------------------------------------------------------
idx: 358082
Subject: RE:Test Results Question
----------------------------------------
Date Sent: 2024-10-21 00:00:00
----------------------------------------
Sender Message:
Hi Dr. Tsao,Thank you for your comments about my blood test results. And I could make an appointment at the quest for A1C test tomorrow afternoon.Best,Ritsuko
----------------------------------------
Provider Response by AFP MYHEALTH POOL:
No response
----------------------------------------
idx: 358081
Subject: Test Results Question
----------------------------------------
Date Sent: 2024-10-18 00:00:00
----------------------------------------
Sender Message:
Hi Dr. Tsao,I got my blood tests results yesterday, and I’m very worried that my Glucose is high. It was slightly outside the normal range for non-fasting, and way outside the normal range for fasting. I didn’t fast but the only thing I ate was oatmeal and eggs and it was about 4 hours before the blood draw so I was very surprised that the glucose level wasn’t closer to the normal fasting range, and I assume it’s related my pancreas function. As you know, I’m very worried about pancreatic cancer.I am eager to get your analysis about it, and any other concerns about other test results, and to move quickly on whatever next step I should take.I am still struggling to eat food and still feeling discomfort after I do eat. Thank you,Ritsuko
----------------------------------------
Provider Response by AFP MYHEALTH POOL:
HI Ritsuko, I have not been able review your results until now and did not see the email that you had sent. Please note, Your results will release immediately, likely before I can review them.  Once I review your results, which may be up to 3-4 business days, I will send comments regarding the results.    We can check another test to make sure your blood sugar average (A1C) is in normal range as your fasting and non-fasting blood sugar can react to when you ate, what you ate, stress, etc.  Your A1C has been ordered and sent electronically to Labcorp to be done non-fasting.  You do not need to have the paper orders, but you can print them out at home through MyHealth (under tests-> ordered tests).  You should try to schedule an appointment for your test at the lab (online or by phone), as they can be busy.  Your results will release immediately, likely before I can review them.  Once I review your results, which may be up to 3-4 business days, I will send comments regarding the results.    Your white blood cell count was slightly low, but improved from your previous which is usually low.  Your thyroid hormone, electrolytes, pancrease enzymes, liver and kidney function are normal.  Your H.pylori stool test is negative for infection.  Please follow up with Dr. Cheng regarding your GI symptoms. Lilian Li-Ning Tsao, MD, 10/21/2024, 8:39 AMI'm not in the office on Mondays, but check my inbox periodically.  
----------------------------------------
2025-06-17 14:10:57,257 - ################################################################################END################################################################################
2025-06-17 14:10:57,279 - ################################################################################START################################################################################
2025-06-17 14:10:57,290 - ✅ similarity: 0.5562
2025-06-17 14:10:57,291 - Sender: MYCHART, GENERIC [ T200162] -> the retrieved similar message : My blood sugar is still high after taking medicine Metformin. In the morning 8.9mol/ mmliter. Converted thats 18 x8.9= 160. After eating it is 14x19=252. Does my medication need to be adjusted to compensate for the high blood sugar?
2025-06-17 14:10:57,291 - Provider's response to this similar message: Hi Yi,Thank you for reaching out. I understand your concern about your blood sugar levels. It seems that your blood sugar is still higher than we would like, even after starting metformin. It may be necessary to adjust your medication or consider additional treatments to better manage your blood sugar levels. I recommend scheduling an appointment so we can discuss this in more detail and make any necessary adjustments. Please use the Stanford MyHealth app to schedule your appointment.In the meantime, continue monitoring your blood sugar levels and follow the dietary recommendations we discussed. Looking forward to seeing you soon.Sujata Behera, PA-C, MA, MPH
2025-06-17 14:10:57,292 - -----------printing the whole thread-------------
2025-06-17 14:10:57,342 - Thread ID: 249144115
--------------------------------------------------------------------------------
idx: 315562
Subject: Non-urgent Medical Question
----------------------------------------
Date Sent: 2024-10-28 00:00:00
----------------------------------------
Sender Message:
My blood sugar is still high after taking medicine Metformin. In the morning 8.9mol/ mmliter. Converted thats 18 x8.9= 160. After eating it is 14x19=252. Does my medication need to be adjusted to compensate for the high blood sugar?
----------------------------------------
Provider Response by AFP MYHEALTH POOL:
Hi Yi,Thank you for reaching out. I understand your concern about your blood sugar levels. It seems that your blood sugar is still higher than we would like, even after starting metformin. It may be necessary to adjust your medication or consider additional treatments to better manage your blood sugar levels. I recommend scheduling an appointment so we can discuss this in more detail and make any necessary adjustments. Please use the Stanford MyHealth app to schedule your appointment.In the meantime, continue monitoring your blood sugar levels and follow the dietary recommendations we discussed. Looking forward to seeing you soon.Sujata Behera, PA-C, MA, MPH
----------------------------------------
2025-06-17 14:10:57,346 - ################################################################################END################################################################################
"""

In [33]:
prompt = f"giving this input {result}, help me format it into a better strucutre so i can copy and paste it into my report "

In [ ]:
gpt_4_1(headers, prompt)